In [1]:
from langchain_mistralai import ChatMistralAI
import getpass
from dotenv import load_dotenv
import os
from langchain_core.prompts import PromptTemplate
import json

In [2]:
load_dotenv()
os.getenv('MISTRAL_API_KEY')

'Je4CrSyBi0EpoJxb0ZGIG7dQhQSWoGSz'

In [3]:
model = ChatMistralAI(model="open-mistral-nemo")

In [4]:
template = """
Given the OCR-extracted information below, return a JSON object where:

1. If the `productname`, `price`, `mfg_date` or `net_weight` is not present, set it to `"NA"`.
2. If `shelf_life` is not provided, set it to `"NA"`.
3. If `price` or `shelf_life` are `"NA"`, predict them from the text or trustable sources and add them to `estimates`.
4. Ensure that `price` is a number (float), `mfg_date` is a date and `shelf_life` is an integer in days.
5. Include all additional details under the `metadata` field.
6. Note that all these products are sold in India.

OCR Extract:
{text}

Expected JSON format:
{{
  "productname": "[Extracted or 'NA']",
  "price": "[Extracted or 'NA']",
  "net_weight": "[Extracted or 'NA']",
  "shelf_life": "[Extracted or 'NA']",
  "mfg_date": "[Extracted or 'NA']",
  "estimates" : {{
    price: "[if price is NA provide an estimated price in numbers]"
    shelf_life: "[if shelf_life is NA provide an estimated price in days]"
  }}
  "metadata": {{
    intuitively find parameters for all other information and put them in here with their specific key words
  }}
}}

Return only the JSON object, without any additional text or explanation.
"""

prompt = PromptTemplate.from_template(template)
chain = prompt | model


In [5]:
input_text = """
ark soy sauce
really tasty fried rice
marinade it adds a punch to remember
WheatAcidity Regulator (INS 260)Preservative INS 202
allergy advice contains wheat soy
may contain nuts sesame seeds mustard milk
keep refrigerated after opening
for sain india nepal bhutan bangladesh only
store in a cool dry and hygienic place
bath no
recipes at wow ching secret com
or
P130408509
in of all taxes
25(R5.0.289
09/05.24(01:47)
a of mfg
usury
0805125
villas
also ghana compound or oshawa a bridge
Lic.No.10013022001865
fora
plaint contact manager consumer
Manutactured byFFortune Foods td.Gut No 256/2
net were
Village Awankhed,Tal-Dindori,Nasik-422202
90g
Lic.No.10013022001948
plan foods div of kaye food industries pvt ltd
Lic.No.10012064000055
890159
Lic.No.10018062000413
things secretive
of capital
818
1102155
trademark
india"""

In [6]:
response = chain.invoke({"text": input_text})
print(response.content)

{
  "productname": "ark soy sauce",
  "price": "NA",
  "net_weight": "90g",
  "shelf_life": "NA",
  "mfg_date": "08/05/24",
  "estimates": {
    "price": 50.0,
    "shelf_life": 30
  },
  "metadata": {
    "allergy_advice": "contains wheat, soy, may contain nuts, sesame seeds, mustard, milk",
    "storage_instructions": "keep refrigerated after opening, store in a cool dry and hygienic place",
    "manufacturer": "Fortune Foods Ltd.",
    "address": "Village Awankhed, Tal-Dindori, Nasik-422202",
    "licenses": ["Lic.No.10013022001865", "Lic.No.10013022001948", "Lic.No.10012064000055", "Lic.No.10018062000413"],
    "country_of_origin": "India"
  }
}


In [9]:
response_content = response.content.strip()
try:
    json_obj = json.loads(response_content)
except json.JSONDecodeError as e:
    print(f"Error decoding JSON: {e}")
    exit(1)

# Define the file path where you want to save the JSON object
file_path = 'output.json'

# Save the JSON object to a file
with open(file_path, 'w') as json_file:
    json.dump(json_obj, json_file, indent=4)

# Print a confirmation message
print(f"JSON object saved to {file_path}")

JSON object saved to output.json
